In [1]:
from ipywidgets import widgets
from ipywidgets import AppLayout
from numpy import asarray
from os.path import isfile, join
from os import listdir
from efficientnet.tfkeras import EfficientNetB0, preprocess_input
from shap.plots._image import colors
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import seaborn as sns
import numpy as np
import pandas as pd
import os
import json
import shap
import cv2
import warnings
import random
import string
import matplotlib.pyplot as pl



In [2]:
df = pd.read_csv("DadosEmbeddingsNormalizados.csv")

In [3]:

scatter = px.scatter(df,
    x="x", y="y",
    marginal_x="histogram", height=500, width=600,
    color="ClasseOriginal", hover_data=['ImageName', df.index], title="Embeddings das imagens")


In [4]:

mypath = f"./valid/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
imagensOrdenadas = sorted(onlyfiles)

#USAR PARA REDIMENSIONAR AS IMAGENS PARA 224X224

def redimensionar(imagem):
  imagem = cv2.resize(imagem, (224, 224),interpolation=cv2.INTER_CUBIC)
  return imagem

for img in onlyfiles:
  imagem = cv2.imread(f'./valid/{img}')
  if imagem.shape[0] != 224 or imagem.shape[1] != 224:
        novaImagem = redimensionar(imagem)
        cv2.imwrite(mypath+"/"+img, novaImagem)
    

In [5]:
model = EfficientNetB0(weights='imagenet')

In [6]:
def image(shap_values, pixel_values=None, labels=None, width=20, aspect=0.2, hspace=0.2, labelpad=None, show=True):
    """ Plots SHAP values for image inputs.
    Parameters
    ----------
    shap_values : [numpy.array]
        List of arrays of SHAP values. Each array has the shap (# samples x width x height x channels), and the
        length of the list is equal to the number of model outputs that are being explained.
    pixel_values : numpy.array
        Matrix of pixel values (# samples x width x height x channels) for each image. It should be the same
        shape as each array in the shap_values list of arrays.
    labels : list
        List of names for each of the model outputs that are being explained. This list should be the same length
        as the shap_values list.
    width : float
        The width of the produced matplotlib plot.
    labelpad : float
        How much padding to use around the model output labels.
    show : bool
        Whether matplotlib.pyplot.show() is called before returning. Setting this to False allows the plot
        to be customized further after it has been created.
    """

    # support passing an explanation object
    if str(type(shap_values)).endswith("Explanation'>"):
        shap_exp = shap_values
        # feature_names = [shap_exp.feature_names]
        # ind = 0
        if len(shap_exp.output_dims) == 1:
            shap_values = [shap_exp.values[..., i] for i in range(shap_exp.values.shape[-1])]
        elif len(shap_exp.output_dims) == 0:
            shap_values = shap_exp.values
        else:
            raise Exception("Number of outputs needs to have support added!! (probably a simple fix)")
        if pixel_values is None:
            pixel_values = shap_exp.data
        if labels is None:
            labels = shap_exp.output_names

    multi_output = True
    if not isinstance(shap_values, list):
        multi_output = False
        shap_values = [shap_values]

    if len(shap_values[0].shape) == 3:
        shap_values = [v.reshape(1, *v.shape) for v in shap_values]
        pixel_values = pixel_values.reshape(1, *pixel_values.shape)

    # make sure labels
    if labels is not None:
        labels = np.array(labels)
        if labels.shape[0] != shap_values[0].shape[0] and labels.shape[0] == len(shap_values):
            labels = np.tile(np.array([labels]), shap_values[0].shape[0])
        assert labels.shape[0] == shap_values[0].shape[0], "Labels must have same row count as shap_values arrays!"
        if multi_output:
            assert labels.shape[1] == len(shap_values), "Labels must have a column for each output in shap_values!"
        else:
            assert len(labels.shape) == 1, "Labels must be a vector for single output shap_values."

    label_kwargs = {} if labelpad is None else {'pad': labelpad}

    # plot our explanations
    x = pixel_values
    fig_size = np.array([3 * (len(shap_values) + 1), 2.5 * (x.shape[0] + 1)])
    if fig_size[0] > width:
        fig_size *= width / fig_size[0]
    fig, axes = pl.subplots(nrows=x.shape[0], ncols=len(shap_values) + 1, figsize=fig_size)
    if len(axes.shape) == 1:
        axes = axes.reshape(1,axes.size)
    for row in range(x.shape[0]):
        x_curr = x[row].copy()

        # make sure we have a 2D array for grayscale
        if len(x_curr.shape) == 3 and x_curr.shape[2] == 1:
            x_curr = x_curr.reshape(x_curr.shape[:2])
        if x_curr.max() > 1:
            x_curr /= 255.

        # get a grayscale version of the image
        if len(x_curr.shape) == 3 and x_curr.shape[2] == 3:
            x_curr_gray = (0.2989 * x_curr[:,:,0] + 0.5870 * x_curr[:,:,1] + 0.1140 * x_curr[:,:,2]) # rgb to gray
            x_curr_disp = x_curr
        elif len(x_curr.shape) == 3:
            x_curr_gray = x_curr.mean(2)

            # for non-RGB multi-channel data we show an RGB image where each of the three channels is a scaled k-mean center
            flat_vals = x_curr.reshape([x_curr.shape[0]*x_curr.shape[1], x_curr.shape[2]]).T
            flat_vals = (flat_vals.T - flat_vals.mean(1)).T
            means = kmeans(flat_vals, 3, round_values=False).data.T.reshape([x_curr.shape[0], x_curr.shape[1], 3])
            x_curr_disp = (means - np.percentile(means, 0.5, (0,1))) / (np.percentile(means, 99.5, (0,1)) - np.percentile(means, 1, (0,1)))
            x_curr_disp[x_curr_disp > 1] = 1
            x_curr_disp[x_curr_disp < 0] = 0
        else:
            x_curr_gray = x_curr
            x_curr_disp = x_curr

        axes[row,0].imshow(x_curr_disp, cmap=pl.get_cmap('gray'))
        axes[row,0].axis('off')
        if len(shap_values[0][row].shape) == 2:
            abs_vals = np.stack([np.abs(shap_values[i]) for i in range(len(shap_values))], 0).flatten()
        else:
            abs_vals = np.stack([np.abs(shap_values[i].sum(-1)) for i in range(len(shap_values))], 0).flatten()
        max_val = np.nanpercentile(abs_vals, 99.9)
        for i in range(len(shap_values)):
            if labels is not None:
                axes[row,i+1].set_title(labels[row,i], **label_kwargs)
            sv = shap_values[i][row] if len(shap_values[i][row].shape) == 2 else shap_values[i][row].sum(-1)
            axes[row,i+1].imshow(x_curr_gray, cmap=pl.get_cmap('gray'), alpha=0.15, extent=(-1, sv.shape[1], sv.shape[0], -1))
            im = axes[row,i+1].imshow(sv, cmap=colors.red_transparent_blue, vmin=-max_val, vmax=max_val)
            axes[row,i+1].axis('off')
    if hspace == 'auto':
        fig.tight_layout()
    else:
        fig.subplots_adjust(hspace=hspace)
    cb = fig.colorbar(im, ax=np.ravel(axes).tolist(), label="SHAP value", orientation="horizontal", aspect=fig_size[0]/aspect)
    cb.outline.set_visible(False)
    if show:
        pl.savefig('imagemExplicada.jpg', bbox_inches='tight')
        #pl.show()

In [7]:
def TransformImageForSHAP(): 
    arrayImagens = []
    i = 0
    for img in onlyfiles:
      imagem = cv2.imread(f'./valid/{img}')
      arrayImagens.append(np.array(imagem.astype('float32')))
      i = i + 1
    return np.array(arrayImagens)

def XAI(dado):

    X = TransformImageForSHAP()

    url = "https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json"
    with open(shap.datasets.cache(url)) as file:
        class_names = [v[1] for v in json.load(file).values()]

    def f(x):
        tmp = x.copy()
        preprocess_input(tmp)
        return model(tmp)


    masker = shap.maskers.Image("inpaint_telea", X[0].shape)

    explainer = shap.Explainer(f, masker, output_names=class_names)


    if int(dado) in df.index:
        nomeDaImagem = df.iloc[[int(dado)]]["ImageName"].values
        if str(nomeDaImagem[0])+'.jpg' in onlyfiles:
            indexQuery = onlyfiles.index(str(nomeDaImagem[0])+'.jpg')
    else:
        print("index não existente !")
        
    shap_values = explainer(X[indexQuery:indexQuery+1], max_evals=100, batch_size=50, outputs=shap.Explanation.argsort.flip[:2])
    image(shap_values)
    

In [8]:
wid_query = {
    "query": widgets.Text(
        value="",
        placeholder="Digite o index da imagem",
        description="",
        disabled=False),
    "button": widgets.Button(
        description='Buscar',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        icon='search', # (FontAwesome names without the `fa-` prefix)
        tooltip='Search for a country'),
    "output": widgets.Image(format='jpg')}

def on_query(button):
    dado = wid_query["query"].value
    XAI(dado)
    file = open("imagemExplicada.jpg", "rb")
    image = file.read()
    wid_query["output"].value = image
    wid_query["output"].height = "500"
    wid_query["output"].width = "500"
    
    
# Eventos
wid_query["button"].on_click(on_query)


AppLayout(
    height="700px",
    header= None,
    left_sidebar= widgets.HBox([go.FigureWidget(scatter)]),
    center= None,
    right_sidebar= widgets.VBox([widgets.Label(value="Entre com um index de uma imagem entre 0 à 498 para visualizar as características de classificação"),widgets.HBox([wid_query["query"], wid_query["button"]]), wid_query["output"]]),
    footer=None)

AppLayout(children=(HBox(children=(FigureWidget({
    'data': [{'customdata': array([['bear_000000000285', 0],…